In [ ]:
from datasets import load_dataset
import json
from tqdm import tqdm 

def convert_mathvision_to_json(split):
    ds = load_dataset("MathLLMs/MathVision")
    
    converted_data = []
    for item in tqdm(ds[split]):
        question_prompt = item['question']
        if item.get('options') and len(item['options'])>0:
            question_prompt += f"The answer is one of the options {', '.join(item['options'])}"
        conversation_entry = {
            "system_prompt": "Solve the following math problem step by step, given the image attached. Write the final answer after <Answer:>",
            "image": item.get("image"),
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>{question_prompt}"
                },
                {
                    "from": "gpt",
                    "value": item.get('answer', 'No answer available.')
                }
            ]
        }
        

        converted_data.append(conversation_entry)
    
    return converted_data

# Convert test split
test_json = convert_mathvision_to_json('test')
with open("/content/drive/MyDrive/reasoning_multimodal_LLMs/example_data/MATHV_test_images.json", 'w') as f:
    json.dump(test_json, f, indent=2)
print(f"Converted {len(test_json)} entries for test split")

# Convert testmini split
testmini_json = convert_mathvision_to_json('testmini')
with open("/content/drive/MyDrive/reasoning_multimodal_LLMs/example_data/MATHV_testmini_images.json", 'w') as f:
    json.dump(testmini_json, f, indent=2)
print(f"Converted {len(testmini_json)} entries for testmini split")